In [1]:
from utils.t5 import *
torch.cuda.empty_cache()
pl.seed_everything(42)

Global seed set to 42


42

In [8]:
torch.cuda.get_device_name(0)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
input_data_name = "preprocessed.csv"                            
data_inpit_dir = "./Data/Preprocessed/"                              
output_dir = "./Data/Model/"
source_column = "outline"    
target_column =  "original_text"          

no_workers = 1

imput_data_path = data_inpit_dir + input_data_name
new_model_name = "d-t5-{}".format(source_column)

In [3]:
data = pd.read_csv(imput_data_path)

data['target_text'] = data[target_column]
data['source_text'] = data[source_column]

data = data[["target_text", "source_text"]]

data['target_len'] = data["target_text"].str.split().str.len()
data['source_len'] = data["source_text"].str.split().str.len()

descriptives = data[['target_len','source_len']].describe()
descriptives

,target_len,source_len
count,300.000000,300.000000
mean,179.040000,145.536667
std,52.512573,63.524798
min,53.000000,1.000000
25%,140.750000,110.000000
50%,179.500000,146.500000
75%,211.750000,183.250000
max,332.000000,319.000000


In [4]:
train, dev_test = train_test_split(data, test_size = 0.3, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [5]:
model = SimpleT5()

In [6]:
model.from_pretrained(model_type="byt5",model_name = "google/byt5-base") # large  "google/mt5-base"

In [7]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

KeyError: 'allocated_bytes.all.current'

In [8]:
model.train(train_df = train, 
            eval_df = dev, 
            source_max_token_len = 319,   
            target_max_token_len = 332,  
            batch_size = 16, max_epochs = 10, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 1,
            new_model_name = new_model_name) # 3

MisconfigurationException: You requested GPUs: [0]
 But your machine only has: []